In [7]:
from dash import Dash, dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
#  from dash_bootstrap_templates import load_figure_template
from dash import dash_table
import plotly.express as px
import pandas as pd


In [9]:
stocks = pd.read_csv("Data/stocks.csv", parse_dates=["Date"], index_col="Date")
stocks.head(2)

,Symbols,Close,High,Low,Open,Volume
Date,,,,,,
2023-01-03,AAPL,125.07,130.90,124.17,130.28,112117471
2023-01-03,MSFT,239.58,245.75,237.40,243.08,25740036


In [10]:
app = Dash(
    __name__, 
    external_stylesheets=[dbc.themes.FLATLY],
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Stock Market Dashboard", className='text-center text-primary mt-3'),
            dcc.DatePickerRange(id = "date_picker", className = "max-auto mb-4"), html.Br()
        ], width=12, style = {'text-align': 'center'})
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id = "single_dropdown", multi = False, searchable = False, className = 'mb-2',           # Sätter man value här så är det förvalt i början. Searchable, kan ej söka i dropdon
                options=[stock for stock in stocks["Symbols"].unique()],
                style = {'color': '#333'}
            ),
            dcc.Graph(id = "volume_graph", config = {'staticPlot': True},            # Staticplot, en av många config options här blir den statisk och ej interaktiv
                figure = {})                              #px.line(stocks, x=stocks.index, y = "Volume", color = "Symbols"))    # Behöver ej finnas här om vi har en callback som sätter den, för alla callbacks anropas när man startar: {} räcker 
        ], xs = 12, sm = 11, md = 10, lg = 5),                                      
        dbc.Col([
                dcc.Dropdown(
                    id = "multi_dropdown", multi = True, className = 'mb-2', value = ["MSFT", "AAPL"],
                    options=[stock for stock in stocks["Symbols"].unique()],
                    style = {'color': '#333'}
                ),
                dcc.Graph(id = "closing_graph", config = {'staticPlot': True},
                    figure = {})                               
            ], xs = 12, sm = 11, md = 10, lg = 5)            # width, här görs den responsiv med threshold. Kan sätta olika värden för olika storlekar
    ], justify = 'evenly'),  
        dbc.Row([
        dbc.Col([
            html.H2("MSFT", id = "data_label"),
            dash_table.DataTable(id="data_table", page_size=15),
        ], width=10)                                                                                                                                   # justerar övrigt mellanrum. Sätts på  kolumnen
    ], justify='center'),                                                                                             # Justify och align sätts på raden
], fluid=True)                                               # Fluid gör att marginaler på sidor försvinner. Sätts på raden


@callback(
    Output("volume_graph", "figure"),                      # Graph och property figure
    Output("data_table", "data"),                          # Tabell och property data
    Output("data_label", "children"),                       # H2 komponent och property children  
    Input("date_picker", "start_date"),                    # Datepicker och property start_date
    Input("date_picker", "end_date"),                      # Datepicker och property end_date
    Input("single_dropdown", "value")                      # Dropdown och value
    #State("data_label", "children")                        #  se dess koppling tll label parameter till nedan funktion och return???? 
)
def single_dropdown_change(start, end, symbol): #label):
    df = stocks.query("Symbols==@symbol").loc[start:end]
    fig = px.line(df, x=df.index, y = "Volume")
    data = df.reset_index().drop(columns =["Symbols"]).to_dict('records')                       # Istället för att ha i col i dash_table.Datatable, eftersom den returnerar det här istället interaktivt. reset gör om index bara till rader 0 och uppåt. Drop = true slänger bort gamla, men om det inte är med så tar den gamla index och gör till kolumn
    return fig, data, symbol #f"{label},{symbol}"         

@callback(
    Output("closing_graph", "figure"), 
    Input("date_picker", "start_date"),                    # Datepicker och property start_date
    Input("date_picker", "end_date"),                      # Datepicker och property end_date                     
    Input("multi_dropdown", "value")
)
def update_closing_graph(start, end, symbols):
    #print(str(symbols))                                            # Kan printa för att se hur det ser ut när man sätter dom (väljer olika alternativ i dropdown)
    if symbols == None : symbols = [""]
    df = stocks.query("Symbols in @symbols").loc[start:end]                       # [stocks["Symbols"].isin(symbols)]
    return px.line(df, x=df.index, y = "Close", color = "Symbols")


app.run(debug=True, jupyter_mode = 'external')




Dash app running on http://127.0.0.1:8050/


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File c:\Users\yonis\.virtualenvs\databehandling-YOUNIS-MOUSSAVI-7D8IQwvr\Lib\site-packages\plotly\express\_chart_types.py:264, in line(
    data_frame=Empty DataFrame
Columns: [Symbols, Close, High, Low, Open, Volume]
Index: [],
    x=DatetimeIndex([], dtype='datetime64[ns]', name='Date', freq=None),
    y='Volume',
    line_group=None,
    color=None,
    line_dash=None,
    symbol=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    text=None,
    facet_row=None,
    facet_col=None,
    facet_col_wrap=0,
    facet_row_spacing=None,
    facet_col_spacing=None,
    error_x=None,
    error_x_minus=None,
    error_y=None,
    error_y_minus=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    orientation=None,
    color_discrete_sequence=None,
    color_discrete_map=N